In [1]:
# Local execution 

import sys 

sys.path.append("connectv2x")

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
%load_ext autoreload

<IPython.core.display.Javascript object>

In [4]:
%autoreload 2

<IPython.core.display.Javascript object>

# Project Description

## Analyzing Impact of V2I connectivity in terms of traffic

In this project the main objective is to understand the impact of vehicle-to-infrastructure (V2I) communication in traffic systems. In particular the focus will be oriented towards the creation of different kind of messages and understanding the effects of those messages in the traffic network.

<img src="assets/img/01_v2i.jpg" alt="drawing" width="600"/>


In order to do that we are going to make use of the package `connectv2x` that includes some basic models and communication for `V2I` systems. For more information about the source code of this package please go [here](https://github.com/aladinoster/connectv2x). Over there you will find a folder called `connectv2x` containing all the models in the submodules `carfollow`, `vehicles`, `demand`, `messages` etc. Some of these will be explored along the module and may help you to progress faster in the project.   

## Contents and structure 

* Exploration and modeling of vehicle dynamics
* Traffic indicators 
* Analyzing different scenarios of V2I messaging 
* Compute emissions from trajectories 
* Scaling up and analyzing deep characteristics. 

Let's begin by importing some plotting libraries that will help us with data visualization

In [5]:
from bokeh.plotting import show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

<IPython.core.display.Javascript object>

## Understanding the basics 

For this application it is considered the simulation of [microscopic traffic models](https://en.wikipedia.org/wiki/Microscopic_traffic_flow_model) where *longitudinal position* follow a specified behavior defined as a function of  two main components, the *headway space* and the *speed differential*. The *car following* behavior describes the behavior of the vehicle in its longitudinal dynamics while the *lane change* behavior describes the behavior in the lateral position. 

In order to modify traffic behavior for a condition, the system is modeled via traffic model where V2I messages modify vehicle speed or lateral position   

#### Traffic model and the fundamental diagram

Before pursuing it is important to understand the value of the fundamental diagram. The fundamental diagram describe the relation ship between ***density*** or concentration of vehicles and the ***flow*** or speed of vehicles. The density is regularly denoted as $\kappa$ for this scenarios as `K`, in the meanwhile the flow is denoted by `Q`. We are going to consider the fact that the relationship follows a piecewise linear behavior as follows:


\begin{aligned}
Q(\kappa) = \begin{cases}
u\kappa\quad 0\leq\kappa\leq \kappa_c\\
w\left(\kappa_x-\kappa\right)\quad \kappa_c\leq\kappa\leq \kappa_x
\end{cases}
\end{aligned}


Let's define the number of vehicles and their initial positions, in order to determine the initial position let's find the minimum spacing. For the moment let's consider the following parameters. `W=5`, `K_X=0.2` and `U=20`. Let's trace the fundamental diagram for this case. We will make use of the `FundamentalDiagram` class in the `traffic` submodule

In [6]:
from connectv2x.traffic import FundamentalDiagram
from bokeh.plotting import show

f = FundamentalDiagram(w=5,u=25,k_x=0.2)
p = f.plot_diagram()
show(p)

<IPython.core.display.Javascript object>

We can easily establish the ***critical density*** which corresponds to the amount of vehicles at which there is maximum flow via

In [7]:
print(f"Capacity: {f.C}")
print(f"Critical density: {f.k_c}")
print(f"Maximum density: {f.k_x}")
print(f"Speed limit: {f.u}")
print(f"Congestion speed: {f.w}")

Capacity: 0.8333333333333334
Critical density: 0.03333333333333333
Maximum density: 0.2
Speed limit: 25
Congestion speed: 5


<IPython.core.display.Javascript object>

The critical density represent the minimum tolerable inter-vehicle distance **headway space** before vehicles start to decrease their speed. On the other hand when the value of density is maximum, and the flow is 0, the vehicles respect the minimum headway space that can be computed from the If the maximum cumulation of vehicles is $\kappa_x$. Then 

\begin{aligned}
\kappa_x = \frac{N}{d} \approx \frac{1}{s_0}  \approx \frac{1}{l_{\text{avg}}+s_{\text{min}}} 
\end{aligned}

By computing the case in the fundamental diagram

In [8]:
print(f"Minimum headway space: {1/f.k_x}")

Minimum headway space: 5.0


<IPython.core.display.Javascript object>

**Q1**: Supose a vehicle has an average length of $l_{\text{avg}}=4$ [m] and the minimum tolerable inter vehicle distance is $s_{\text{min}}=2.25$[m]. Consider the congestion wave speed $w=6.25$ [m/s] and a free flow speed of $u=25$ [m/s]. Compute and plot the fundamental diagram for this case and find the value of the capacity and minimum headway space:

<span style= "color:blue">Provide your answers below:</span>

#### Car following behavior and traffic model 

Keep in mind the constants `K_X`, `W`, `U` you have computed, they will be useful for building up the behavior of cars in traffic. 

For the sake of clarity, the following corresponds to the notations for variable description in the model. It is considered the vehicle position of a vehicle as $x_n$ and the headway space between a vehicle and its leader as $s_n = x_{n-1}-x_{n}$. The vehicle's speed and acceleration are defined as $v_n$,$a_n$ respectively. The operator $\Delta v_n = v_{n-1} - v_{n}$ refers to the difference between the leading vehicle and the following one. 

For a determined vehicle in the network the longitudinal dynamics are determined by the acceleration behavior. In this case it is considred Tampere's Law. 

$$ 
a_n(t+T_n) = \min \left(c_{1,n-1}\Delta v_{n-1,n} + 
c_{2}\left(\Delta x_{n,n-1} - \left(s_0+\tau v_n(t)\right)\right),
c_{3}\left(v^\star(t) - v_n(t)\right)\right)
$$

One of the main features of this model is the adaptability to a specific speed condition, while preserving properties of the traffic such as the car following behavior in congestion situation. This feature makes it possible to trace features in the fundamental diagram. 

*To implement the model a `class` object called `Tampere` has been implemented. The class intends to describe the full behavior of the vehicle.* 

#### Parameters 

So far parameters in the model have been fixed although random scenarios can be also considered.

| Parameter     | Value     | Units |
:--------------:|:---------:|:------:
$$c_1,c_2,c_3$$ | 0.5       |
$$\tau$$        | $$\frac{1}{wk_x}$$ | [s]
$$w$$           | $$6.25$$  | [m/s]
$$k_x$$         | $$0.16$$  | [veh/km]
$$u_i$$         | $$25$$    | [m/s]

Please follow step by step variable definitions for more detail into simulations. Let define the parameters declared in the table as `C_1,C_2,C_3,W,K_X,W` and import the class `Tampere` from the submodule `connectv2x.carfollow`. 

In [9]:
from connectv2x.carfollow import Tampere

C_1, C_2, C_3 = (0.5,0.5,0.5)

<IPython.core.display.Javascript object>

In the following we are going to compute the initial speeds `V_0`, positions `X_0` for simulating the vehicle's dynamic. Let's consider for the moment a set of two vehicles 

In [11]:
import numpy as np 

N_VEH = 2 # Number of vehicles 
X0 = np.array([f.s_x,0])
V0 = np.ones(N_VEH) * f.u
V_CLASS = ["HDV", "HDV"]

veh_list = []
Tampere.reset()
for x0, v0, vtype in zip(X0, V0, V_CLASS):
    veh_list.append(Tampere(x0=x0, v0=v0, veh_type=vtype))
print(f"List of vehicles: {veh_list}")

List of vehicles: [Tampere(x0=5.0,v0=25.0), Tampere(x0=0.0,v0=25.0)]


<IPython.core.display.Javascript object>

Based on the definition of the fundamental diagram let's compute the parameters required to evolve the acceleration equation. In general in order to evolve the acceleration equation it is required to solve the following dynamical system: 


\begin{aligned}
x_n(t+T) &= x_n(t) + v_n(t)T\\
v_n(t+T) &= v_n(t) + a_n(t) T\\
\end{aligned}

In order to evolve this set of equations in time it is required to account for the initial positions for all vehicles and in particular for a particular definition of $a_0(t)$ the first vehicle in the formation. 

In [13]:
# Minimum spacing 

from connectv2x.support import speed_pulse

def lead_spd(x):
    """  Leader's function to control speed drop in space 
         Speed Drop: 20 m/s 
         Position: 5 Km
         Duration: 20 Km
    """
    return speed_pulse(x, drop=20, delay=5000, duration=20000)

<IPython.core.display.Javascript object>

In [8]:
import numpy as np

<IPython.core.display.Javascript object>

**Q1**: Create a function that considers as an inputs:

* `x_leader`
* `v_leader`
* `x_ego`
* `v_ego` 

In [ ]:
def acceleration(x_leader,v_leader, ):
    # C1 * 
    return a

In [2]:
from connectv2x.carfollow import Tampere, W_I, U_I, K_X, A_MIN, A_MAX
from connectv2x.support import speed_pulse
from connectv2x.messages import Msg1, Msg2
import numpy as np
import pandas as pd
import datetime as dt
import csv 

from connectv2x.plottools import plot_single_trace, plot_xva, plot_histogram, plot_multiple_trajectories
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook, export_png
from bokeh.layouts import row, column

output_notebook()

Loading BokehJS ...

#### Hyper parameter variation for sensitivity analysis

* `MPR`: Market penetration rate [0,1]
* `Q_PERC`: Percentage of maximum flow (0,1]
* `MIN_DIST`: Minium distance for message broadcasting [4000,14000]

In [ ]:
# Parameters
MPR = 0.4
MIN_DIST = 10000
Q_PERC = 1


## Input definitions 

Main input definitions for the simulation such as the `simulation time`, `flow`, `distance to congestion`, `congestion length`, `perception radious`

**Important note**: To run simulations be sure to start always from the beginning

### <span style="color:red">Define parameters below </span>

<span style= "color:red">See comments for more details </span>

* simulation time: `time`
* works position: `X_CONGESTION`
* works length: `L_CONGESTION`
* radious of messages: `PERCEP_RADIOUS`

In [ ]:
# Constant values
N = 100  # Number of vehicles to simulate
T_TOTAL = 960  # Simulation time [s]
time = np.arange(T_TOTAL)  # Time vector

# Traffic characteristics
X_CONGESTION = 15000  # Position of congestion in space [m]
L_CONGESTION = 1500  # Approximate congestion length in space [m]

# Messages for V2V
SPEED_REDUCTION = 5.5  # Amount of speed reduction [m/s]
PERCEP_RADIOUS = 3000  # Radious of perception of the broadcasted messages [m]

## Vehicle definitions 

In the following cell content regarding the vehicle will be defined in particular the `flow`. The current capacity value is: 



### <span style="color:red">Define parameters below </span>

<span style= "color:red">See comments for more details </span>

* Amount of flow w.r.t to capacity: `Q_PERC`

In [ ]:
# Capacity
C = (U_I * W_I * K_X) / (W_I + U_I)
# print(f"Capacity value per lane: {C*3600} [veh/h]")

# Vehicle Initial position / speed
TF = C * Q_PERC

In [ ]:
# Vehicle initializer
X0 = np.flip(np.arange(0, N) * (W_I + U_I) / (W_I * K_X) * 1 / Q_PERC)
V0 = np.ones(N) * U_I
A0 = np.zeros(N)

veh_list = []

np.random.seed(42)  # Reproducibility
ID_CAV = np.random.randint(1, N - 1, int(N * MPR))  # Id Connected Vehicles
D_CLASS = {k: "CAV" for k in ID_CAV}
V_CLASS = [D_CLASS.get(i, "HDV") for i in range(N)]  # All vehicle types

# Initializing vehicles
Tampere.reset()
for x0, v0, vtype in zip(X0, V0, V_CLASS):
    veh_list.append(Tampere(x0=x0, v0=v0, l0=0, veh_type=vtype))

# Setting leader for vehicle i
for i in range(1, N):
    veh_list[i].set_leader(veh_list[i - 1])

ID_CAVN = [i for i, j in enumerate(V_CLASS) if j == "CAV"]

## Message characteristics 

The following cell computes the distances at which the messages are going to be broadcasted 

In [ ]:
# Scenario conditions
D_ACCEPT = X_CONGESTION - 1000  # Broad casting messages @ 14Km
D_ACCEPT = D_ACCEPT - np.random.exponential(PERCEP_RADIOUS, N * 1000)
D_ACCEPT = D_ACCEPT[(D_ACCEPT > MIN_DIST) & (D_ACCEPT < X_CONGESTION)]
D_ACCEPT = np.random.choice(D_ACCEPT, N)

# accept_dist = plot_histogram(D_ACCEPT,"Acceptane Distance [Km]")
# show(accept_dist)

## Message information 

The following is just to illustrate the message generation and definitions for the simulation 

In [ ]:
# Road works speed profile
def lead_spd(x):
    """  Leader's function to control speed drop in space 
         Speed Drop: 20 m/s 
         Position: 15 Km
         Duration: 20 Km
    """
    return speed_pulse(x, drop=20, delay=X_CONGESTION, duration=L_CONGESTION)

# x_t = np.linspace(0, 20000, 20000)
# v_t = lead_spd(x_t)
# leader_xt = plot_single_trace(
#      x_t, v_t, "Leaders' speed", "Space [m]", "Speed [m/s]"
# )
# show(leader_xt)

## Main simulation

The following code defines the simulation that is run. Execute the code directly in case you want to modify the message change `Msg1` by `Msg2`

In [ ]:
# Dynamical evalution
X = X0
V = V0
A = A0

send_message = Msg2  # Msg2 # Defines the type of message to be send

d_accept = X_CONGESTION - 1000
msg_fix = send_message(d_accept)

for t in time:
    for veh in veh_list:
        if veh.type == "CAV" and not veh.acc:
            d_accept = D_ACCEPT[veh.idx]
            msg = send_message(d_accept)
            veh.register_control_speed(msg)
        elif veh.type == "HDV" and not veh.acc:
            veh.register_control_speed(msg_fix)

        veh.step_evolution(control=lead_spd)

    V = np.vstack((V, np.array([veh.v for veh in veh_list])))
    X = np.vstack((X, np.array([veh.x for veh in veh_list])))
    A = np.vstack((A, np.array([veh.a for veh in veh_list])))

V = V[1:, :]
X = X[1:, :]
A = A[1:, :]

# Indicators 

The following produces the total travel time for vehicles

In [ ]:
df_x = pd.DataFrame(X)
now = dt.datetime(2019,11,26,13,0) # This is to set a special day for synchronous purposes 
delta = dt.timedelta(seconds=1)
time_vector = [now+n*delta for n in range(T_TOTAL)]
df_x.index = time_vector 

def find_travel_time(df):
    time_entry  = min(df[df>df_x.iloc[0,0]].index) # minimum time after leader entrance 
    time_exit = max(df[df<X_CONGESTION+L_CONGESTION].index) # maximum time before exit
    travel_time = time_exit-time_entry
    return travel_time.total_seconds()

ttt = df_x.apply(find_travel_time)

tt_mean,tt_var = np.mean(ttt),np.std(ttt) 
ttt = sum(ttt)
# tthist = plot_histogram(ttt)
# show(tthist)

We then compute indicators for security, time to collision: 

**Notes**:
* The model is simple and considers no acceleration from current speed 
* If vehicles are separating instead of colliding then the value of time will be negative hence we discard it 
* In cases vehicles are in equilibrium the time to collision is `inf` so we also discard those samples. 

In [ ]:
# Time to colission

df_v = pd.DataFrame(V)
df_v.index = time_vector 
df_ttc = -df_x.diff(axis=1)/df_v.diff(axis=1)
df_ttc = df_ttc.replace([np.inf, -np.inf], np.nan)
ttc = df_ttc[df_ttc>0].mean(axis=1,skipna=True).mean(axis=0,skipna=True)

# Due to some computational issues time to collision will be avoided instead we compute 

Computing spacing for safety characteristics in the network

In [ ]:
# Average Space headway 
df_s = -df_x.diff(axis=1).mean(axis=1,skipna=True)

# Plots and results 

The following are extracts of some plots and results to be analyzed 

In [ ]:
# Creating plots
x_t = time
v_t = V[:, 0]
leader_vt = plot_single_trace(x_t, v_t, "Leaders' speed", "Time [s]", "Speed [m/s]")
x_t = time
a_t = A[:, 0]
leader_at = plot_single_trace(x_t, a_t, "Leaders' acceleration", "Time [m]", "Acceleration [m/s²]")
zooms = ((MIN_DIST, X_CONGESTION + L_CONGESTION), (-1, U_I + 1), (A_MIN - 0.5, A_MAX + 0.5))
titles = (
    f"X-Time MPR={MPR*100}% F={TF*3600}[veh/h] D={MIN_DIST}[m]",
    f"V-Time MPR={MPR*100}% F={TF*3600}[veh/h] D={MIN_DIST}[m]",
    f"X-Time MPR={MPR*100}% F={TF*3600}[veh/h] D={MIN_DIST}[m]"
  )
pos, spd, acc = plot_xva(time, X, V, A, y_range=zooms, titles=titles)
poswoz = plot_multiple_trajectories(time, X, V, titles[0], "Time [secs]", "Position [m]")
# show(leader_at)

# Exporting files

The following are extracts of some plots and results to be analyzed 

In [ ]:
# Writting CSV File for Phem 

df_X = pd.DataFrame(X)
df_X = df_X.melt(var_name="id",value_name="x")
df_X = df_X.reset_index()
df_X = df_X.rename(columns={'index': 'time',"id":"vehicle number"})
df_X["y"] = 0
df_v = pd.DataFrame(V)
df_V = df_v.melt(var_name="id",value_name="v")
df_X["speed"]=df_V["v"]*3.6 
df_X["road inclination"] = 0 
df_X["vehicle type"] = 100
df_X["segment number"] = 0
correct_order = [0,2,3,1,4,5,6,7]
df_X = df_X[df_X.columns[correct_order]]
data = "data/csv/"
filename = f"phem_mpr-{MPR}_q-{Q_PERC}_d-{MIN_DIST}.csv"
df_X.to_csv(data+filename)

In [ ]:
# Writting CSV File for Spacing 
filename = f"spacing_mpr-{MPR}_q-{Q_PERC}_d-{MIN_DIST}.csv"
data = "data/csv/"
df_s.to_csv(data+filename)

In [ ]:
# Writting trajectories file
data = "data/"
filename = f"img/cases/position/pos_mpr-{MPR}_q-{Q_PERC}_d-{MIN_DIST}.png"
export_png(pos, filename=data + filename)
filename = f"img/cases/position/poswoz_mpr-{MPR}_q-{Q_PERC}_d-{MIN_DIST}.png"
export_png(poswoz, filename=data + filename)
print(f"File: {filename} has been saved")
filename = f"img/cases/speed/spd_mpr-{MPR}_q-{Q_PERC}_d-{MIN_DIST}.png"
export_png(spd, filename=data + filename)
print(f"File: {filename} has been saved")
filename = f"img/cases/acceleration/acc_mpr-{MPR}_q-{Q_PERC}_d-{MIN_DIST}.png"
export_png(acc, filename=data + filename)
print(f"File: {filename} has been saved")

In [ ]:
# Writting indicators file 

csvTitle = ['mpr','q','distance','meanTT','stdTT','totalTT','TTC']
csvData = [MPR,TF,MIN_DIST,tt_mean,tt_var,ttt,ttc]
print(f"MPR:{MPR},Q:{TF},D:{MIN_DIST}")

try:
    with open('data/Indicators.csv','a') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerow(csvData)
except FileNotFoundError:
    with open('data/Indicators.csv','w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerow(csvTitle)
        writer.writerow(csvData)
        

A. Ladino